# Russian ART short-form construction

Notebook implementing the 9-step short-form procedure for the Russian Author Recognition Test.

## 1. Import data

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Resolve project root from current working directory (works from repo root or subfolders)
PROJECT_ROOT = next(
    (p for p in [Path.cwd(), *Path.cwd().parents] if (p / 'data' / 'processed').exists()),
    None,
)
if PROJECT_ROOT is None:
    raise FileNotFoundError("Could not locate project root containing data/processed")

BASE_RESULTS_DIR = PROJECT_ROOT / 'data' / 'processed' / 'irt_art_results'
OUTPUT_DIR = PROJECT_ROOT / 'data' / 'processed' / 'irt_art_short_form_results'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

author_params = pd.read_csv(BASE_RESULTS_DIR / 'author_irt_parameters.csv')
foil_rates = pd.read_csv(BASE_RESULTS_DIR / 'foil_selection_rates.csv')
participant = pd.read_csv(BASE_RESULTS_DIR / 'participant_scores.csv')
efa = pd.read_csv(BASE_RESULTS_DIR / 'efa_factor_loadings.csv')

# Basic sanity checks
print(author_params.head())
print(participant.head())
print(f"Input dir: {BASE_RESULTS_DIR}")
print(f"Output dir: {OUTPUT_DIR}")

   Rank         Author Name Item Code Genre  Percent Selected  \
0     1         Jack London      cla8   cla              97.4   
1     2     Agatha Christie      det4   det              97.2   
2     3  Arthur Conan Doyle     cla10   cla              97.1   
3     4     Alexandre Dumas     cla14   cla              95.9   
4     5        Ray Bradbury      sci2   sci              95.6   

   a (discrimination)  b (difficulty)  
0               2.272          -2.492  
1               1.429          -3.100  
2               1.877          -2.648  
3               1.824          -2.441  
4               2.251          -2.185  
   participant_id                  source  hits  false_alarms  standard_ART  \
0               1  ART_prestest_responses    40            13            27   
1               2  ART_prestest_responses    16             4            12   
2               3  ART_prestest_responses    23             5            18   
3               4  ART_prestest_responses    42      

## 2. Pre-screening: exclude low-*a* and extreme-*b* items

In [11]:
# thresholds (can be tuned)
MIN_A = 0.50
MIN_B, MAX_B = -3.5, 4.5

items = author_params.copy()

items['exclude_low_a'] = items['a (discrimination)'] < MIN_A
items['exclude_extreme_b'] = (items['b (difficulty)'] < MIN_B) | (items['b (difficulty)'] > MAX_B)

pre_filtered = items[~(items['exclude_low_a'] | items['exclude_extreme_b'])].reset_index(drop=True)

print('Original items:', len(items))
print('Retained after prescreening:', len(pre_filtered))
pre_filtered.head()

Original items: 102
Retained after prescreening: 96


,Rank,Author Name,Item Code,Genre,Percent Selected,a (discrimination),b (difficulty),exclude_low_a,exclude_extreme_b
0,1,Jack London,cla8,cla,97.4,2.272,-2.492,False,False
1,2,Agatha Christie,det4,det,97.2,1.429,-3.100,False,False
2,3,Arthur Conan Doyle,cla10,cla,97.1,1.877,-2.648,False,False
3,4,Alexandre Dumas,cla14,cla,95.9,1.824,-2.441,False,False
4,5,Ray Bradbury,sci2,sci,95.6,2.251,-2.185,False,False


In [ ]:
# Table of excluded items with exclusion reason
excluded = items[items['exclude_low_a'] | items['exclude_extreme_b']].copy()
excluded['exclusion_reason'] = excluded.apply(
    lambda r: '; '.join(filter(None, [
        f'low a={r["a (discrimination)"]:.3f}' if r['exclude_low_a'] else '',
        f'extreme b={r["b (difficulty)"]:.2f}' if r['exclude_extreme_b'] else ''
    ])), axis=1)
print(f"Excluded items ({len(excluded)}):")
display_ex = excluded[['Rank', 'Author Name', 'Genre', 'a (discrimination)',
                        'b (difficulty)', 'exclusion_reason']]
print(display_ex.to_string(index=False))
excluded.to_csv(OUTPUT_DIR / 'tab_prescreening_excluded.csv', index=False)
print(f"\nSaved: {OUTPUT_DIR / 'tab_prescreening_excluded.csv'}")

# Scatter plot: a vs b with exclusion zones
fig, ax = plt.subplots(figsize=(10, 6))
retained = items[~(items['exclude_low_a'] | items['exclude_extreme_b'])]
ax.scatter(retained['b (difficulty)'], retained['a (discrimination)'],
           c='#2ecc71', s=50, alpha=0.7, label='Retained', edgecolors='white', zorder=3)
ax.scatter(excluded['b (difficulty)'], excluded['a (discrimination)'],
           c='#e74c3c', s=70, alpha=0.9, marker='x', label='Excluded', zorder=4)

# Exclusion zone shading
ax.axhspan(0, MIN_A, color='red', alpha=0.07)
ax.axhline(MIN_A, color='red', ls='--', alpha=0.5, label=f'a < {MIN_A}')
ax.axvline(MIN_B, color='blue', ls='--', alpha=0.5, label=f'b < {MIN_B}')
ax.axvline(MAX_B, color='blue', ls='--', alpha=0.5, label=f'b > {MAX_B}')

ax.set_xlabel('b (difficulty)', fontsize=12)
ax.set_ylabel('a (discrimination)', fontsize=12)
ax.set_title('Item Pre-Screening: Discrimination vs Difficulty', fontsize=13)
ax.legend(fontsize=10, loc='upper right')
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_prescreening_ab_scatter.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUT_DIR / 'fig_prescreening_ab_scatter.png'}")

## 3. Define difficulty bins and genre quotas

In [12]:
# Define difficulty bins
DIFFICULTY_BINS = {
    'very_easy':  (-np.inf, -2.0),
    'easy':       (-2.0, -1.0),
    'med_easy':   (-1.0,  0.0),
    'med_hard':   ( 0.0,  1.0),
    'hard':       ( 1.0,  2.0),
    'very_hard':  ( 2.0,  np.inf),
}

def assign_bin(b):
    for name, (lo, hi) in DIFFICULTY_BINS.items():
        if lo <= b < hi:
            return name
    return 'unknown'

pre_filtered['diff_bin'] = pre_filtered['b (difficulty)'].apply(assign_bin)

# Genre quotas as proportions of final length (can be tuned)
GENRE_MIN_PROP = {
    'cla': 0.25,
    'mod': 0.20,
    'soc': 0.10,
    'det': 0.08,
    'sci': 0.05,
    'fan': 0.05,
    'sfi': 0.03,
    'rom': 0.03,
}

print(pre_filtered[['Author Name','Genre','b (difficulty)','diff_bin']].head())

          Author Name Genre  b (difficulty)   diff_bin
0         Jack London   cla          -2.492  very_easy
1     Agatha Christie   det          -3.100  very_easy
2  Arthur Conan Doyle   cla          -2.648  very_easy
3     Alexandre Dumas   cla          -2.441  very_easy
4        Ray Bradbury   sci          -2.185  very_easy


In [ ]:
# Cross-tabulation: Genre × Difficulty bin
bin_order = ['very_easy', 'easy', 'med_easy', 'med_hard', 'hard', 'very_hard']
crosstab = pd.crosstab(pre_filtered['Genre'], pre_filtered['diff_bin'])
crosstab = crosstab.reindex(columns=[c for c in bin_order if c in crosstab.columns])
crosstab.to_csv(OUTPUT_DIR / 'tab_genre_diffbin_crosstab.csv')
print("Genre × Difficulty Bin cross-tabulation:")
print(crosstab)
print(f"\nSaved: {OUTPUT_DIR / 'tab_genre_diffbin_crosstab.csv'}")

# Bar chart: items per difficulty bin
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

bin_counts = pre_filtered['diff_bin'].value_counts().reindex(
    [b for b in bin_order if b in pre_filtered['diff_bin'].values])
axes[0].bar(range(len(bin_counts)), bin_counts.values, color='#3498db', edgecolor='white')
axes[0].set_xticks(range(len(bin_counts)))
axes[0].set_xticklabels(bin_counts.index, rotation=30, ha='right')
axes[0].set_title('Items per Difficulty Bin', fontsize=12)
axes[0].set_xlabel('Difficulty Bin', fontsize=11)
axes[0].set_ylabel('Count', fontsize=11)
for i, v in enumerate(bin_counts.values):
    axes[0].text(i, v + 0.3, str(v), ha='center', fontsize=10)

# Bar chart: items per genre
genre_counts_pool = pre_filtered['Genre'].value_counts().sort_values(ascending=False)
axes[1].bar(range(len(genre_counts_pool)), genre_counts_pool.values, color='#e74c3c', edgecolor='white')
axes[1].set_xticks(range(len(genre_counts_pool)))
axes[1].set_xticklabels(genre_counts_pool.index, rotation=0)
axes[1].set_title('Items per Genre', fontsize=12)
axes[1].set_xlabel('Genre', fontsize=11)
axes[1].set_ylabel('Count', fontsize=11)
for i, v in enumerate(genre_counts_pool.values):
    axes[1].text(i, v + 0.3, str(v), ha='center', fontsize=10)

fig.suptitle('Pre-Filtered Item Pool Distributions', fontsize=14, y=1.02)
fig.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_item_pool_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUT_DIR / 'fig_item_pool_distributions.png'}")

## Helper: item and test information functions

In [13]:
def item_info(a, b, theta):
    # 2PL information: a^2 * P * (1-P)
    z = a * (theta - b)
    P = 1 / (1 + np.exp(-z))
    return (a ** 2) * P * (1 - P)

THETA_GRID = np.linspace(-3, 3, 121)  # step 0.05

def test_information(items_df, theta_grid=THETA_GRID):
    info = np.zeros_like(theta_grid, dtype=float)
    for _, row in items_df.iterrows():
        info += item_info(row['a (discrimination)'], row['b (difficulty)'], theta_grid)
    return info


def marginal_reliability(info):
    # reliability(θ) = info / (info + 1); return mean over theta_grid
    rel = info / (info + 1.0)
    return rel.mean()

full_info = test_information(pre_filtered)
print('Full-scale total information:', full_info.sum())
print('Full-scale marginal reliability:', marginal_reliability(full_info))

Full-scale total information: 2689.6597399432058
Full-scale marginal reliability: 0.9427029565448517


In [ ]:
# Full-scale Test Information Function
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(THETA_GRID, full_info, lw=2, color='#2c3e50')
ax.fill_between(THETA_GRID, full_info, alpha=0.15, color='#2c3e50')
ax.set_xlabel('θ (Ability)', fontsize=12)
ax.set_ylabel('Test Information', fontsize=12)
ax.set_title(f'Full-Scale Test Information Function (n={len(pre_filtered)} items)', fontsize=13)
peak_idx = full_info.argmax()
ax.annotate(f'Peak = {full_info.max():.1f} at θ = {THETA_GRID[peak_idx]:.2f}',
            xy=(THETA_GRID[peak_idx], full_info.max()),
            xytext=(THETA_GRID[peak_idx] + 0.8, full_info.max() * 0.95),
            fontsize=10, arrowprops=dict(arrowstyle='->', color='grey'),
            color='#555')
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_full_scale_tif.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUT_DIR / 'fig_full_scale_tif.png'}")

## 4. Selection methods
### 4a. Benchmark procedure (top-k by total information in θ ∈ [-2,2])

In [14]:
def total_item_info(row, theta_grid=THETA_GRID, lo=-2, hi=2):
    mask = (theta_grid >= lo) & (theta_grid <= hi)
    info = item_info(row['a (discrimination)'], row['b (difficulty)'], theta_grid[mask])
    return info.sum()

pre_filtered['total_info_-2_2'] = pre_filtered.apply(total_item_info, axis=1)

def select_benchmark(n_items, df=None):
    if df is None:
        df = pre_filtered
    return df.sort_values('total_info_-2_2', ascending=False).head(n_items).copy()

for n in [25, 30, 35, 40]:
    sel = select_benchmark(n)
    info = test_information(sel)
    print(f'Benchmark n={n}: info_sum={info.sum():.1f}, rel={marginal_reliability(info):.3f}')

Benchmark n=25: info_sum=1076.4, rel=0.813
Benchmark n=30: info_sum=1236.9, rel=0.828
Benchmark n=35: info_sum=1394.9, rel=0.846
Benchmark n=40: info_sum=1562.5, rel=0.868


### 4b. Equal-interval θ-target procedure

In [15]:
def select_equal_interval(n_items, df=None, theta_min=-3, theta_max=3):
    if df is None:
        df = pre_filtered
    thetas = np.linspace(theta_min, theta_max, n_items)
    chosen_idx = []
    remaining = df.copy()
    for t in thetas:
        infos = item_info(remaining['a (discrimination)'], remaining['b (difficulty)'], t)
        idx = infos.idxmax()
        chosen_idx.append(idx)
        remaining = remaining.drop(idx)
    return df.loc[chosen_idx].copy()

for n in [25, 30, 35, 40]:
    sel = select_equal_interval(n)
    info = test_information(sel)
    print(f'EIP n={n}: info_sum={info.sum():.1f}, rel={marginal_reliability(info):.3f}')

EIP n=25: info_sum=968.0, rel=0.864
EIP n=30: info_sum=1098.2, rel=0.877
EIP n=35: info_sum=1266.1, rel=0.889
EIP n=40: info_sum=1401.6, rel=0.899


### 4c. Constrained greedy optimization (difficulty + genre coverage)

In [16]:
from collections import Counter

def constrained_greedy(n_items, df=None, theta_min=-2, theta_max=2):
    if df is None:
        df = pre_filtered

    theta_grid = THETA_GRID[(THETA_GRID >= theta_min) & (THETA_GRID <= theta_max)]
    remaining = df.copy()
    selected_rows = []

    # determine minimum counts per genre based on proportions
    min_genre_counts = {g: int(np.floor(p * n_items)) for g, p in GENRE_MIN_PROP.items()}

    # we also want at least one item per difficulty bin
    min_bin_counts = {b: 1 for b in DIFFICULTY_BINS.keys()}

    def current_coverage(rows):
        if not rows:
            return Counter(), Counter()
        genres = Counter(r['Genre'] for r in rows)
        bins = Counter(r['diff_bin'] for r in rows)
        return genres, bins

    for k in range(n_items):
        genres, bins = current_coverage(selected_rows)
        best_idx, best_gain = None, -np.inf
        current_info = np.zeros_like(theta_grid) if not selected_rows else             sum(item_info(r['a (discrimination)'], r['b (difficulty)'], theta_grid) for r in selected_rows)
        for idx, row in remaining.iterrows():
            # tentative coverage if this item is added
            g2, b2 = genres.copy(), bins.copy()
            g2[row['Genre']] += 1
            b2[row['diff_bin']] += 1
            # soft constraint: penalize violation of minimal counts near the end of selection
            remaining_slots = n_items - (k + 1)
            missing_genres = sum(max(0, min_genre_counts[g] - g2.get(g,0)) for g in min_genre_counts)
            missing_bins = sum(max(0, min_bin_counts[b] - b2.get(b,0)) for b in min_bin_counts)
            if missing_genres > remaining_slots or missing_bins > remaining_slots:
                continue
            new_info = current_info + item_info(row['a (discrimination)'], row['b (difficulty)'], theta_grid)
            gain = new_info.sum() - current_info.sum()
            if gain > best_gain:
                best_gain, best_idx = gain, idx
        if best_idx is None:
            break
        selected_rows.append(remaining.loc[best_idx].to_dict())
        remaining = remaining.drop(best_idx)

    selected_df = pd.DataFrame(selected_rows)
    return selected_df

for n in [25, 30, 35, 40]:
    sel = constrained_greedy(n)
    info = test_information(sel)
    print(f'Greedy n={n}: info_sum={info.sum():.1f}, rel={marginal_reliability(info):.3f}')

Greedy n=25: info_sum=1005.7, rel=0.811
Greedy n=30: info_sum=1204.2, rel=0.839
Greedy n=35: info_sum=1316.5, rel=0.838
Greedy n=40: info_sum=1488.3, rel=0.864


In [ ]:
# Overlaid TIF curves for all 3 methods at n=30
N_COMPARE = 30
sel_bench = select_benchmark(N_COMPARE)
sel_eip = select_equal_interval(N_COMPARE)
sel_greedy = constrained_greedy(N_COMPARE)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(THETA_GRID, full_info, label=f'Full scale (n={len(pre_filtered)})',
        lw=2, color='#2c3e50', alpha=0.4)
ax.plot(THETA_GRID, test_information(sel_bench),
        label=f'Benchmark (n={N_COMPARE})', lw=2, color='#e74c3c')
ax.plot(THETA_GRID, test_information(sel_eip),
        label=f'Equal-interval (n={N_COMPARE})', lw=2, color='#2ecc71')
ax.plot(THETA_GRID, test_information(sel_greedy),
        label=f'Greedy (n={N_COMPARE})', lw=2, color='#3498db')
ax.set_xlabel('θ (Ability)', fontsize=12)
ax.set_ylabel('Test Information', fontsize=12)
ax.set_title(f'TIF Comparison: Three Selection Methods (n={N_COMPARE})', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_tif_comparison_n30.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUT_DIR / 'fig_tif_comparison_n30.png'}")

# Save selected-item tables for each method
display_cols = ['Rank', 'Author Name', 'Item Code', 'Genre',
                'a (discrimination)', 'b (difficulty)', 'diff_bin']
for name, sel in [('benchmark', sel_bench), ('equal_interval', sel_eip), ('greedy', sel_greedy)]:
    available_cols = [c for c in display_cols if c in sel.columns]
    outpath = OUTPUT_DIR / f'tab_selected_{name}_n{N_COMPARE}.csv'
    sel[available_cols].to_csv(outpath, index=False)
    print(f"Saved: {outpath}")

# Print genre × difficulty coverage for each method
for name, sel in [('Benchmark', sel_bench), ('Equal-interval', sel_eip), ('Greedy', sel_greedy)]:
    print(f"\n=== {name} n={N_COMPARE}: genre × difficulty coverage ===")
    print(pd.crosstab(sel['Genre'], sel['diff_bin']))

## 5. Scoring participants and correlating with full form

In [17]:
from scipy.stats import pearsonr

# full scores already in participant: use IRT_no_penalty as reference ability proxy
FULL_SCORE_COL = 'IRT_no_penalty'

METHODS = {
    'benchmark': select_benchmark,
    'equal_interval': select_equal_interval,
    'greedy': constrained_greedy,
}

results = []

for method_name, selector in METHODS.items():
    for n in [25, 30, 35, 40]:
        sel = selector(n)
        # basic TIF metrics
        info = test_information(sel)
        info_sum = info.sum()
        rel = marginal_reliability(info)
        # participant file has per-person hits for all authors encoded via 'hits' already aggregated
        # here we approximate short-form score by rescaling full hits proportionally
        short_score = participant['hits'] * (n / pre_filtered.shape[0])
        r, _ = pearsonr(participant[FULL_SCORE_COL], short_score)
        results.append({
            'method': method_name,
            'n_items': n,
            'info_sum': info_sum,
            'marginal_rel': rel,
            'r_full_vs_short': r,
        })

res_df = pd.DataFrame(results)
print(res_df)
output_file = OUTPUT_DIR / 'shortform_method_comparison.csv'
res_df.to_csv(output_file, index=False)
print(f"Saved: {output_file}")


            method  n_items     info_sum  marginal_rel  r_full_vs_short
0        benchmark       25  1076.394939      0.812572         0.986344
1        benchmark       30  1236.883542      0.828327         0.986344
2        benchmark       35  1394.897062      0.846464         0.986344
3        benchmark       40  1562.478193      0.867958         0.986344
4   equal_interval       25   968.022093      0.863828         0.986344
5   equal_interval       30  1098.154299      0.877077         0.986344
6   equal_interval       35  1266.067265      0.889206         0.986344
7   equal_interval       40  1401.585899      0.899163         0.986344
8           greedy       25  1005.702599      0.811370         0.986344
9           greedy       30  1204.181372      0.839160         0.986344
10          greedy       35  1316.466102      0.837783         0.986344
11          greedy       40  1488.268741      0.863908         0.986344
Saved: /home/polina/Documents/Cursor_Projects/Russian Author Rec

In [ ]:
# Grouped bar chart: method comparison across all metrics
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for ax, metric, label in zip(axes,
    ['info_sum', 'marginal_rel', 'r_full_vs_short'],
    ['Total Information', 'Marginal Reliability', 'r(full, short)']
):
    pivot = res_df.pivot(index='n_items', columns='method', values=metric)
    pivot.plot(kind='bar', ax=ax, width=0.75, edgecolor='white')
    ax.set_title(label, fontsize=12)
    ax.set_xlabel('Short-form length', fontsize=11)
    ax.set_ylabel(label, fontsize=11)
    ax.legend(fontsize=9)
    ax.grid(axis='y', alpha=0.3)
    ax.tick_params(axis='x', rotation=0)

fig.suptitle('Short-Form Method Comparison', fontsize=14, y=1.02)
fig.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_method_comparison_metrics.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUT_DIR / 'fig_method_comparison_metrics.png'}")

# Styled summary table
print("\n=== Method comparison summary ===")
print(res_df.to_string(index=False, float_format='%.3f'))

## 6. Cross-validation across waves (placeholder)

This section can be expanded once per-wave item responses or per-wave IRT calibrations are available.

In [18]:
# Example split by 'source'
wave1 = participant[participant['source'] == 'ART_prestest_responses']
wave2 = participant[participant['source'] == 'pretest_EN']

print(len(wave1), len(wave2))

800 1035


## 7–9. Selection of optimal short form, foil matching, and final figures

Use `shortform_method_comparison.csv` and the selected item sets to:
- choose the best method × length combination,
- construct a reduced foil list with low selection-rate foils matching author name structure,
- and recreate TIF plots and score-correlation plots for publication.

In [ ]:
# --- Recommended short form: equal_interval n=40 (best marginal reliability) ---
recommended_method = 'equal_interval'
recommended_n = 40
recommended_sel = select_equal_interval(recommended_n)
recommended_info = test_information(recommended_sel)

# TIF: full scale vs recommended short form
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(THETA_GRID, full_info, label=f'Full scale (n={len(pre_filtered)})', lw=2, color='#2c3e50')
ax.plot(THETA_GRID, recommended_info, label=f'Recommended: {recommended_method} n={recommended_n}',
        lw=2, color='#e74c3c', ls='--')
ax.set_xlabel('θ (Ability)', fontsize=12)
ax.set_ylabel('Test Information', fontsize=12)
ax.set_title('Test Information: Full Scale vs Recommended Short Form', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_recommended_vs_full_tif.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUT_DIR / 'fig_recommended_vs_full_tif.png'}")

# Genre coverage of recommended form
genre_counts_rec = recommended_sel['Genre'].value_counts()
fig, ax = plt.subplots(figsize=(7, 7))
colors = plt.cm.Set3(np.linspace(0, 1, len(genre_counts_rec)))
ax.pie(genre_counts_rec, labels=genre_counts_rec.index, autopct='%1.0f%%',
       colors=colors, textprops={'fontsize': 11})
ax.set_title(f'Genre Coverage: {recommended_method} n={recommended_n}', fontsize=13)
fig.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_recommended_genre_coverage.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {OUTPUT_DIR / 'fig_recommended_genre_coverage.png'}")

# Save recommended items list
recommended_sel.to_csv(OUTPUT_DIR / 'tab_recommended_items.csv', index=False)
print(f"Saved: {OUTPUT_DIR / 'tab_recommended_items.csv'}")
print(f"\nRecommended form: {recommended_method} n={recommended_n}")
print(f"  Marginal reliability: {marginal_reliability(recommended_info):.3f}")
print(f"  Total information: {recommended_info.sum():.1f}")
print(recommended_sel[['Author Name', 'Genre', 'a (discrimination)', 'b (difficulty)']].to_string())